# **MLOps avec MLFlow**

<img src='https://www.databricks.com/sites/default/files/mlflow.png'>

Dans ce notebook, nous allons utiliser la bibliothèque Mlflow pour suivre les expériences de machine learning.

Nous allons :
- créer un projet Mlflow
- créer une expérience
- créer des runs
- suivre les métriques, les paramètres et les artefacts
- visualiser les résultats dans l'interface Mlflow
- enregistrer un modèle puis le charger dans un autre notebook

In [ ]:
# Installation des dépendances
!pip install mlflow==2.16.1 torchvision boto3


# Démarrage du server mlflow 
# mlflow server

  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached graphql_core-3.2.6-py3-none-any.whl.metadata (11 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
   ------------------ --------------------- 12.6/26.7 MB 65.5 MB/s eta 0:00:01
   ------------------------------------ --- 24.1/26.7 MB 61.0 MB/s eta 0:00:01
   ---------------------------------------- 26.7/26.7 MB 49.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   --

  You can safely remove it manually.


In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

from torch import float16
model_name = 'gp2'

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2",).save_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2").save_pretrained(model_name)


In [ ]:
import mlflow
import os 

os.environ['AWS_ACCESS_KEY_ID'] = '...'
os.environ['AWS_SECRET_ACCESS_KEY'] = '...'

mlflow.set_tracking_uri("https://inrae-mlflow-server-6c1614130433.herokuapp.com/")

# Configuration d'une expérience (création si elle n'existe pas) : mlflow.set_experiment("Crédit Agricol")
experiment = mlflow.set_experiment('INRAE - Kevin Duranty')

2025/03/12 07:48:40 INFO mlflow.tracking.fluent: Experiment with name 'INRAE - Kevin Duranty' does not exist. Creating a new experiment.


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="GPT2"):
    
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Log des métriques
    mlflow.log_metric("train_score", 0.567 )


    # Créer une pipeline pour la génération de texte
    text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    # Log du modèle en précisant le type de tâche
    mlflow.transformers.log_model(
        transformers_model=text_generation_pipeline,
        artifact_path="model",
        task="text-generation",
        registered_model_name='GPT2'
    )
    print("Modèle sauvegardé et logué avec MLflow !")


Device set to use cpu
2025/03/12 07:49:23 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to 404 Client Error. (Request ID: Root=1-67d12e72-2a96f13b7e919647559e354a;15226e19-ff12-4f25-963e-f49f444f2e2f)

Repository Not Found for url: https://huggingface.co/gp2/resolve/main/README.md.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
2025/03/12 07:49:23 WARNING mlflow.transformers: Unable to find license information for this model. Please verify permissible usage for the model you are storing prior to use.
2025/03/12 07:49:23 WARNING mlflow.utils.environment: On Windows, timeout is not supported for model requirement inference. Therefore, the operation is not bound by a timeout and may hang indefinitely. If it hangs, please consider specifying the signature manually.
Successfully registered model 'GPT2'.
2025/03/12 07:51:04 INFO mlflow.store.model

Modèle sauvegardé et logué avec MLflow !


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from torch import float16


with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="GPT2"):
    model_name = "gpt2"
    mlflow.log_artifact(model_name)

2025/03/11 20:00:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run llama3-1B at: https://inrae-mlflow-server-6c1614130433.herokuapp.com/#/experiments/1/runs/75e803e735384d49b2feb1252a564d65.
2025/03/11 20:00:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://inrae-mlflow-server-6c1614130433.herokuapp.com/#/experiments/1.


In [24]:
text_generation_pipeline("Bonjour")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Bonjour: As you've seen from the videos, there are some things that will keep people away. For instance, if you've got a guy just hanging out and you don't want him to be able to stop and ask him for directions"}]

In [25]:
import mlflow

logged_model = 'runs:/71489f463ec2425fae384b36e79e9ae3/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

Device set to use cpu


In [43]:
loaded_model.predict('Bonjour')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Bonjour, France. We may be proud of being the youngest and fattest city in Europe; but if we were now grown-up, we should be ashamed to be part of them—but where are we going to go? —']

In [29]:


mlflow.artifacts.download_artifacts(
    artifact_uri="s3://inrae-mlflow/storage/2/71489f463ec2425fae384b36e79e9ae3/artifacts/model",
    dst_path="./downloaded_model"
)


'c:\\Users\\Quera\\Desktop\\INRAE-Llama3-main\\downloaded_model\\model'

In [ ]:
model = AutoModelForCausalLM.from_pretrained('downloaded_model/model/model')
tokenizer = AutoTokenizer.from_pretrained('downloaded_model/model/components/tokenizer')

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [42]:
text_generation_pipeline('Bonjour')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Bonjour, the name is used because it is very well known for its sour fruit and its flavour. It is commonly mentioned in the UK as having a sour flavour for good health reasons. It often has been regarded as an excellent substitute for plain'}]